In [5]:
import cv2
import os
import torch
import glob
import yt_dlp

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()
def download_video_from_youtube(url, save_path):
    print(f" Downloading video from: {url}")
    ydl_opts = {
        'outtmpl': save_path,
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'merge_output_format': 'mp4',
        'quiet': False
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print(f" Video downloaded and saved as: {save_path}")
def detect_and_compress(input_path, output_path, snapshot_dir):
    os.makedirs(snapshot_dir, exist_ok=True)

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f" Failed to open video: {input_path}")
        return
    print(" Video opened successfully.")
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) // 2
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) // 2
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
    frame_num = 0
    suspicious_log = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_num += 1
        results = model(frame)
        detections = results.xyxy[0]
        names = model.names
        suspicious = False
        for *box, conf, cls in detections:
            x1, y1, x2, y2 = map(int, box)
            label = names[int(cls)]

            if label == "person":
                color = (0, 255, 0)
                text = "Normal Person"
            else:
                color = (0, 0, 255)
                text = "Suspicious"
                suspicious = True
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        if suspicious:
            snap_path = os.path.join(snapshot_dir, f"suspicious_frame_{frame_num}.jpg")
            cv2.imwrite(snap_path, frame)
            suspicious_log.append((frame_num, frame_num / fps))
        resized = cv2.resize(frame, (w, h))
        out.write(resized)
        cv2.imshow(" Compressed Detection", resized)
        if cv2.waitKey(1) & 0xFF == 27:
            print(" Detection stopped by user.")
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("\n Suspicious frames with non-person objects:")
    for f, t in suspicious_log:
        print(f" Frame {f} at {t:.2f} seconds")
def show_snapshots(folder):
    images = sorted(glob.glob(os.path.join(folder, "*.jpg")))
    if not images:
        print(" No suspicious snapshots found.")
        return
    print(f" Showing {len(images)} suspicious snapshots. Press any key for next, ESC to exit.")
    for img_path in images:
        img = cv2.imread(img_path)
        cv2.imshow("Suspicious Snapshot", img)
        if cv2.waitKey(0) & 0xFF == 27:
            break
    cv2.destroyAllWindows()
def main():
    yt_url = "https://www.youtube.com/watch?v=9_mhedrKICs"
    local_video_path = "input_downloaded.mp4"
    output_video = "output_compressed.mp4"
    snapshot_dir = "snapshots"
    if not os.path.exists(local_video_path):
        download_video_from_youtube(yt_url, local_video_path)
    print("\n Starting detection and compression...")
    detect_and_compress(local_video_path, output_video, snapshot_dir)
    print("\n Showing suspicious snapshots...")
    show_snapshots(snapshot_dir)
if __name__ == "__main__":
    main()


Using cache found in C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-5-20 Python-3.12.4 torch-2.7.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):



 Starting detection and compression...
 Video opened successfully.


C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:907: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\manas/.cache\torch\hub\ultralytics_yolov5_master\models

 Detection stopped by user.

 Suspicious frames with non-person objects:

 Showing suspicious snapshots...
 Showing 513 suspicious snapshots. Press any key for next, ESC to exit.
